The aim of this notebook is to make use of the word2vec model to find similar songs

In [4]:
import pandas as pd
import numpy as np
import gensim.models.word2vec as w2v
import multiprocessing
import os
import re
import pprint
import sklearn.manifold
import matplotlib.pyplot as plt

In [3]:
!pip install smart_open.gcs

  ERROR: Could not find a version that satisfies the requirement smart_open.gcs (from versions: none)
ERROR: No matching distribution found for smart_open.gcs


Though non english artists were removed, the dataset contained Hindi lyrics of Lata Mangeshkar written in English. Therefore, I decided to remove all songs sung by her.

In [5]:
songs = pd.read_csv("hhgroups_merge_28_05.csv", header=0, encoding='utf-8')
#songs.head()
songs = songs[songs.artista != 'Lata Mangeshkar']
songs.head()

,id,artista,cancion,album,letra,anyo,visitas
0,0,Denom,Machete (con Jarfaiter y Gente jodida),Medicina,"Para su nuevo disco ""Medicina"", Denom ha vuelt...",2019,126
1,1,Denom,Vacío (con Ivo Incuerdo),Medicina,"[Denom]\nYo que quería, yo que pedía vida,\nSe...",2019,361
2,2,Denom,El orgullo es fiel (con Juancho Marqués y Elio...,Medicina,"""El orgullo es fiel"" es uno de los cortes incl...",2019,262
3,3,Denom,Mueve mueve (con Fernandocosta),Medicina,"[Estribillo: Denom] (x2)\nMueve, mueve, mueve,...",2019,578
4,4,Jaro Desperdizio,Insomnia,"Sin álbum, es un vídeo suelto","[Estribillo]\nY en esta noche, ¿Quién me arrop...",2019,219


To train the word2vec model, we first need to build its vocabulary. To do that, I iterated over each song and added it to an array that can later be fed to the model.

In [6]:
text_corpus = []
for song in songs['letra']:
    words = song.lower().split()
    text_corpus.append(words)


# Dimensionality of the resulting word vectors.
#more dimensions, more computationally expensive to train
#but also more accurate
#more dimensions = more generalized
num_features = 50
# Minimum word count threshold.
min_word_count = 1

# Number of threads to run in parallel.
#more workers, faster we train
num_workers = multiprocessing.cpu_count()

# Context window length.
context_size = 7


downsampling = 1e-1

# Seed for the RNG, to make the results reproducible.
#random number generator
#deterministic, good for debugging
seed = 1

songs2vec = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

songs2vec.build_vocab(text_corpus)
print (len(text_corpus))

9325


In [7]:
import time
start_time = time.time()



songs2vec.train(text_corpus, total_examples=songs2vec.corpus_count, epochs=2)

if not os.path.exists("trained"):
    os.makedirs("trained")

songs2vec.save(os.path.join("trained", "songs2vectors.w2v"))

print("--- %s seconds ---" % (time.time() - start_time))

--- 43.76863217353821 seconds ---


In [8]:
songs2vec = w2v.Word2Vec.load(os.path.join("trained", "songs2vectors.w2v"))

#### Let's explore our model

Find similar words

In [9]:
songs2vec.wv.most_similar("viajar")

[('descansar', 0.9716689586639404),
 ('sonreir,', 0.9706870317459106),
 ('quedarte', 0.9673882722854614),
 ('volverme', 0.9667607545852661),
 ('encontrarte,', 0.966421902179718),
 ('encontrarme', 0.9645244479179382),
 ('despedirme', 0.964408278465271),
 ('andar,', 0.9634292721748352),
 ('arder', 0.9628416895866394),
 ('descansar,', 0.9623440504074097)]

In [10]:
songs2vec.wv.most_similar("vacaciones")

[('comprando', 0.9712678790092468),
 ('diciembre', 0.9691998958587646),
 ('catorce', 0.9688772559165955),
 ('nadando', 0.9671924114227295),
 ('playas', 0.9669414758682251),
 ('pegando', 0.9665864706039429),
 ('millas', 0.9651497602462769),
 ('leña', 0.9635815024375916),
 ('chocar', 0.9635589122772217),
 ('disparando', 0.9633252620697021)]

Words out of context

In [11]:
songs2vec.wv.doesnt_match("te quiero".split())

C:\Users\merxe\Anaconda3\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning:

arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.



'te'

In [12]:
songs2vec.most_similar(positive=['mujer', 'rey'], negative=['hombre'])


C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning:

Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).



[('fan', 0.8521075248718262),
 ('hija', 0.8484303951263428),
 ('fallaste,', 0.8441058993339539),
 ('guapa', 0.840936005115509),
 ('cerda', 0.8408082723617554),
 ('sales', 0.8375235199928284),
 ('pose', 0.8365832567214966),
 ('gorra', 0.8359953761100769),
 ('bastardo', 0.8321945667266846),
 ('liga,', 0.8308533430099487)]

Semantic distance between words

In [13]:
def nearest_similarity_cosmul(start1, end1, end2):
    similarities = songs2vec.wv.most_similar_cosmul(
        positive=[end2, start1],
        negative=[end1]
    )
    start2 = similarities[0][0]
    print("{0} es a {1}, lo que {2} es a {3}".format(start1, end1, start2, end2))

In [14]:
nearest_similarity_cosmul("españa", "mar", "tristeza")

españa es a mar, lo que peña es a tristeza


With the word vector embeddings in place, it is now time to calculate the normalised vector sum of each song. This process can take some time since it has to be done for each of 57,000 songs.

In [15]:
print(songs2vec['noche'])
def songVector(row):
    vector_sum = 0
    words = row.lower().split()
    for word in words:
        vector_sum = vector_sum + songs2vec[word]
    vector_sum = vector_sum.reshape(1,-1)
    normalised_vector_sum = sklearn.preprocessing.normalize(vector_sum)
    return normalised_vector_sum


import time
start_time = time.time()

songs['song_vector'] = songs['letra'].apply(songVector)




C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-p

[ 0.5777398  -1.0693552  -0.24910359 -0.4651037  -0.4018697   0.08697096
 -0.4719546   0.49643192  0.18414113  0.42977467  0.3251425   0.13983874
  0.33680525  0.29282048  0.15473433  0.11474808 -0.77683187  0.11107488
 -0.8303292   0.16958566  0.74934363 -0.01551843  0.45895553 -0.21401891
  0.00569714 -0.7182337  -0.26178446 -1.1162214   0.49269974 -0.4081639
  0.303292   -0.4123504  -0.31022656 -0.78211975  0.38174847  0.38645858
  0.7035189  -0.8330429  -0.03493417  0.8017993   0.06610047  0.5217789
 -0.02439773  0.77416223  0.15954432 -0.4003032   0.3195451  -0.4154366
 -0.44895765  0.04045037]


C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-p

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-p

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-p

...

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-p

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-p

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-p

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-p

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

C:\Users\merxe\Anaconda3\lib\site-p

**t-sne and random song selection** 

The songs have 50 dimensions each. Application of t-sne is memory intensive and hence it is slightly easier on the computer to use a random sample of the 57,000 songs.

In [17]:
song_vectors = []
from sklearn.model_selection import train_test_split

train, test = train_test_split(songs, test_size = 0.9)


for song_vector in train['song_vector']:
    song_vectors.append(song_vector)

train.head(10)

,id,artista,cancion,album,letra,anyo,visitas,song_vector
6876,26,Piezas y Jayder,Disbalans,Melancholia,Os pongo a barrer la migaja del mantel mientra...,2015,2811,"[[0.13888302, -0.17777452, -0.1258073, -0.0429..."
7886,46,Billy el niño y Don Dinero,Otra historia de coria,Otra historia de Coria,"Hey, hey,\nEsto una historia de Coria,\nCon Bi...",2009,1558,"[[0.13576506, -0.18208686, -0.13587151, -0.021..."
5028,48,Falsalarma,Musas,La memoria de mis pasos,"[El Santo]\nYa sé que nada de esto es fácil,\n...",2018,1079,"[[0.14964692, -0.18073803, -0.1333018, -0.0511..."
7832,47,Los chikos del maíz,Tú al gulag y yo a California,La estanquera de Saigón,[Nega]\nMis J-Haybber de barrio contra tus Nik...,2014,3310,"[[0.14169396, -0.1669525, -0.114747345, -0.024..."
7910,15,Saunah,No somos iguales,Superviviente,"[Saunah]\nNadie me dijo que esto fuera fácil, ...",2014,1153,"[[0.120064266, -0.17381865, -0.122278534, -0.0..."
8347,12,Sr. Zambrana,Por favor,Nudismo en estéreo,"Sonia era un ciclón\npura pasión, macho\nme co...",2014,966,"[[0.14820938, -0.18556035, -0.13482407, -0.049..."
3579,4,Legendario,Elias Maluko (con Shotta),Sin álbum,"[Shotta]:Yeee,Shotta,Legendario\n[Legendario]:...",0,2449,"[[0.13179524, -0.16555963, -0.12089864, -0.032..."
885,5,Lom,C,Proyecto renacer,"Amor platónico, amor platónico...\nAmor platón...",2011,15737,"[[0.15418616, -0.16275436, -0.12145792, -0.037..."
2427,7,MC Kla,Ahi Estare Yo,Sin álbum,"TE AMO\nMK FOR LIF3!!!\n\n\nKLA, Blooding, Mc ...",2008,2089,"[[0.13399331, -0.21070741, -0.14110139, -0.061..."
3906,1,Cartel de santa,Todas mueren por mí,Sin álbum,[Estribillo]\nY es que es así...\ntodas mueren...,0,3161,"[[0.1408872, -0.18213966, -0.13499455, -0.0338..."


I had a fairly measly 4gb machine and wasn't able to generate a more accurate model. However, one can play around with the number of iterations, learning rate and other factors to fit the model better. If you have too many dimensions (~300+), it might make sense to use PCA first and then t-sne.

In [18]:
X = np.array(song_vectors).reshape((932, 50))

start_time = time.time()
tsne = sklearn.manifold.TSNE(n_components=2, n_iter=250, random_state=0, verbose=2)

all_word_vectors_matrix_2d = tsne.fit_transform(X)

print("--- %s seconds ---" % (time.time() - start_time))

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 932 samples in 0.004s...
[t-SNE] Computed neighbors for 932 samples in 0.117s...
[t-SNE] Computed conditional probabilities for sample 932 / 932
[t-SNE] Mean sigma: 0.028298
[t-SNE] Computed conditional probabilities in 0.080s
[t-SNE] Iteration 50: error = 75.0271301, gradient norm = 0.3392347 (50 iterations in 1.275s)
[t-SNE] Iteration 100: error = 75.3125992, gradient norm = 0.3453411 (50 iterations in 1.188s)
[t-SNE] Iteration 150: error = 75.3485107, gradient norm = 0.3348270 (50 iterations in 1.013s)
[t-SNE] Iteration 200: error = 76.0776138, gradient norm = 0.3177303 (50 iterations in 1.364s)
[t-SNE] Iteration 250: error = 75.4984894, gradient norm = 0.3083665 (50 iterations in 1.232s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 75.498489
[t-SNE] KL divergence after 251 iterations: 17976931348623157081452742373170435679807056752584499659891747680315726078002853876058955863276687817154045895351438246

In [19]:
df=pd.DataFrame(all_word_vectors_matrix_2d,columns=['X','Y'])

df.head(10)

train.head()

df.reset_index(drop=True, inplace=True)
train.reset_index(drop=True, inplace=True)

Joining two dataframes to obtain each song's corresponding X,Y co-ordinate.

In [20]:
two_dimensional_songs = pd.concat([train, df], axis=1)

two_dimensional_songs.head()

,id,artista,cancion,album,letra,anyo,visitas,song_vector,X,Y
0,26,Piezas y Jayder,Disbalans,Melancholia,Os pongo a barrer la migaja del mantel mientra...,2015,2811,"[[0.13888302, -0.17777452, -0.1258073, -0.0429...",0.023904,-0.411230
1,46,Billy el niño y Don Dinero,Otra historia de coria,Otra historia de Coria,"Hey, hey,\nEsto una historia de Coria,\nCon Bi...",2009,1558,"[[0.13576506, -0.18208686, -0.13587151, -0.021...",0.186840,-0.509192
2,48,Falsalarma,Musas,La memoria de mis pasos,"[El Santo]\nYa sé que nada de esto es fácil,\n...",2018,1079,"[[0.14964692, -0.18073803, -0.1333018, -0.0511...",-0.675398,-5.732510
3,47,Los chikos del maíz,Tú al gulag y yo a California,La estanquera de Saigón,[Nega]\nMis J-Haybber de barrio contra tus Nik...,2014,3310,"[[0.14169396, -0.1669525, -0.114747345, -0.024...",0.003222,-0.279789
4,15,Saunah,No somos iguales,Superviviente,"[Saunah]\nNadie me dijo que esto fuera fácil, ...",2014,1153,"[[0.120064266, -0.17381865, -0.122278534, -0.0...",0.241759,0.209821


**Plotting the results**

Using plotly, I plotted the results so that it becomes easier to explore similar songs based on their colors and clusters.

In [21]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

import plotly.graph_objs as go

trace1 = go.Scatter(
    y = two_dimensional_songs['Y'],
    x = two_dimensional_songs['X'],
    text = two_dimensional_songs['artista'],
    mode='markers',
    marker=dict(
        size= 5,#'7',
        color = np.random.randn(5717), #set color equal to a variable
        colorscale='Viridis',
        showscale=True
    )
)
data = [trace1]

iplot(data)

  Created wheel for retrying: filename=retrying-1.3.3-cp37-none-any.whl size=11435 sha256=89889fdd75d4d2c12d1b2ae3f99160431aa91cff897aa12da6a27e22443eaa97
  Stored in directory: C:\Users\merxe\AppData\Local\pip\Cache\wheels\d7\a9\33\acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying


In [23]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure(data=go.Scatter(
    y = two_dimensional_songs['Y'],
    x = two_dimensional_songs['X'],
    text = two_dimensional_songs['artista']+ "_"+two_dimensional_songs['cancion'] ,
    mode='markers',
    marker=dict(
        size= 10,#'7',
        color = np.random.randn(5717), #set color equal to a variable
        colorscale='Viridis',
        showscale=True
    )
))

fig.show()

In [0]:
## LOOK FOR COMMON SONGS AND ANALYZE THE TEXT

In [24]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure(data=go.Scatter(
    y = np.random.randn(500),
    mode='markers',
    marker=dict(
        size=16,
        color=np.random.randn(500), #set color equal to a variable
        colorscale='Viridis', # one of plotly colorscales
        showscale=True
    )
))

fig.show()

In [25]:
import plotly.express as px
df = px.data.iris()
fig = px.scatter_3d(df, x='sepal_length', y='sepal_width', z='petal_width',
              color='species')
fig.show()